<h2><b><i>Radial Basis Function Neural Network</i></b></h2>

Σκοπός της παρούσας άσκησης είναι η κατηγοριοποίηση χειρόγραφων ψηφίων από το [MNIST dataset](https://www.wikiwand.com/en/MNIST_database) με τη χρήση νευρωνικών δικτύων [RBF](https://en.wikipedia.org/wiki/Radial_basis_function) (RBF). Για την κατηγοροποίηση των χειρόγραφων ψηφίων, χρησιμοποιείται ο αλγόριθμος K-means. Στη κλασική υλοποίηση του K-means όπου έχουμε το κέντρο του cluster και ταξινομομούμε κάθε σημείο με βάση την απόσταση από τα κέντρα. Σε αυτή την περίπτωση μπορεί να υπάρξουν σημεία που να μην μπορούν να αντιστοιχηθούν σε κάποιο cluster. Με τη χρήση του νευρωνικού δικτύου RBF κάθε σημείο ταξινομείται σε ομάδες με συγκεκριμένο κέντρο, γνωρίζοντας όμως τις αποστάσεις μεταξύ των κέντρων αλλά και με διαστήματα εμπιστοσύνης ως προς την πρόβλεψή μας. Με αυτό το τρόπο όλα τα σημεία αντιστοιχίζονται σε ένα cluster. Για να είναι ομαλή η ταξινόμηση των στοιχείων στα clusters μπορεί να χρησιμοποιηθεί μια εκθετική συνάρτηση υψωμένη στην αρνητική τιμή της απόστασης ως γινόμενο με έναν βαθμωτό συντελεστή beta, όπως φαίνεται παρακάτω:

![alt text](https://miro.medium.com/max/145/1*MIay3aIlpT18yewOfnvTiQ.png)

Στα πλαίσια του παραδείγματος για το β χρησιμοποιήθηκε ο παρακάτω τύπος

![alt text](https://miro.medium.com/max/608/1*YEcI_P6orY917fQrzHQEjQ.png)

Όπου k ο αριθμός των clusters και Dmax η μέγιστη ευκλείδια απόσταση μεταξύ των κέντρων. Το dataset που θα χρησιμοποιήσετε είναι διαθέσιμο [εδώ](https://drive.google.com/file/d/1WrVPB3C1vIApWNz5BaSuNmHFopf5htCg/view?usp=sharing) (θα πρέπει να το φορτώσετε στο Colab για να το τρέξετε). 

*Οι ερωτήσεις της άσκησης είναι μαζί με το αντίστοιχο βήμα στον κώδικα*


**Βήμα 1ο**

Τρέχετε τον κώδικα στον οποίο και ορίζονται οι συναρτήσεις που θα μας χρειαστούν. Στα πλαίσια της άσκησης χρησιμοποιούμε μία παραμετροποιημένη έκδοση της συνάρτησης K-Means, όπου μας επιστρέφει τα κέντρα των ομάδων (clusters) καθώς και την τυπική απόκλιση τους.

In [20]:
"""''

Copyright © 2020 Tarlan Ahadli

Permission is hereby granted, free of charge, to any person obtaining a
copy of this software and associated documentation files (the “Software”),
to deal in the Software without restriction, including without limitation
the rights to use, copy, modify, merge, publish, distribute, sublicense,
and/or sell copies of the Software, and to permit persons to whom the Software
is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included
in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY
KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE
WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE
AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT
HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY,
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR
OTHER DEALINGS IN THE SOFTWARE.

''"""

import numpy as np


def get_distance(x1, x2):
    sum = 0
    for i in range(len(x1)):
        sum += (x1[i] - x2[i]) ** 2
    return np.sqrt(sum)


def kmeans(X, k, max_iters):
    centroids = X[np.random.choice(range(len(X)), k, replace=False)]
    # centroids = [np.random.uniform(size=len(X[0])) for i in range(k)]

    converged = False
    current_iter = 0

    while (not converged) and (current_iter < max_iters):

        cluster_list = [[] for i in range(len(centroids))]

        for x in X:  # Go through each data point
            distances_list = []
            for c in centroids:
                distances_list.append(get_distance(c, x))
            cluster_list[int(np.argmin(distances_list))].append(x)

        cluster_list = list((filter(None, cluster_list)))

        prev_centroids = centroids.copy()

        centroids = []

        for j in range(len(cluster_list)):
            centroids.append(np.mean(cluster_list[j], axis=0))

        pattern = np.abs(np.sum(prev_centroids) - np.sum(centroids))

        print('K-MEANS: ', int(pattern))

        converged = (int(pattern) == 0)

        current_iter += 1

    return np.array(centroids), [np.std(x) for x in cluster_list]


**Βήμα 2ο**

Τρέχουμε την κλάση για το RBFNN

---

***Ερώτηση 1***

Να αναφέρετε επιγραμματικά με ποιον τρόπο λειτουργεί ένα νευρωνικό δίκτυο RBF.

***Ερώτηση 2***

Στα πλαίσια του παραδείγματος, για κάθε αριθμό (κλάση) έχουμε μία μόνο ομάδα (cluster). Τι θα συνέβαινε αν είχαμε παραπάνω από μία ομάδες που θα αντιστοιχούσαν σε μία κλάση; Πώς λύνεται αυτό μέσω του RBF;


**Απάντηση 1**

Ένα νευρωνικό δίκτυο RBF έχει ένα μόνο κρυφό στρώμα που αποτελείτε από νευρώνες. Οι νευρώνες αυτοί έχουν σαν χαρακτηριστικά, τα δικά τους centroids και την δικία τους συνάρτηση την λεγόμενη Radial Basis Function που εξαρτάται από τα centroids και στην περίπτωση μας είναι αυτή που αναφέρεται στην εκφώνηση. H συνάρτηση αυτή ποσοτικοποιεί την ομοιότητα ενός σημείου με το centroid και εκφράζει μια πιθανότητα να ανήκει στο cluster του συγκεκριμένου centroid. Συνεπώς αν λάβουμε αυτή τη πιθανότητα από όλους τους νευρώνες , κάθε ένας από αυτούς αντιπροσωπεύει και ενα centroid, τότε μπορούμε να αποφανθούμε και σε ποιο cluster ανήκει. Η διαδικασία που θα αποκτήσουμε τα centroids είναι συνήθως μέσω του k-means αλγόριθμου και στη συνέχεια η εκπαίδευση θα ανανεώσει τη συνεισφορά κάθε centroid-cluster σε μια συγκεκριμένη κλάση του dataset μας.  

**Απάντηση 2**

Οι νευρώνες του κρυφού στρώματος του RBF συνδέονται με output νευρώνες με βάρη. Στόχος των βαρών αυτών ειναι να προσδιορίσουν την συνεισφορά της ομοιότητας κάθε cluster με μία κλάση , συνεπώς αν ένα σημείο έχει πολύ μεγάλη ομοιότητα με ένα centroid-cluster υπάρχει το ενδεχόμενο να κριθεί μη σημαντική η συνεισφορά τou cluster. Έτσι θα αντιμετωπίσει το RBFNN δύο ανταγωνιστικά clusters. Ο προσδιορισμός της συνεισφοράς είναι η διαδικασία του training που έχουμε ανανέωση των βαρών.


In [21]:
class RBF:

    def __init__(self, X, y, tX, ty, num_of_classes,
                 k, std_from_clusters=True):
        self.X = X
        self.y = y

        self.tX = tX
        self.ty = ty

        self.number_of_classes = num_of_classes
        self.k = k
        self.std_from_clusters = std_from_clusters

    def convert_to_one_hot(self, x, num_of_classes):
        arr = np.zeros((len(x), num_of_classes))
        for i in range(len(x)):
            c = int(x[i])
            arr[i][c] = 1
        return arr

    def get_rbf(self, x, c, s):
        distance = get_distance(x, c)
        return 1 / np.exp(-distance / s ** 2)

    def get_rbf_as_list(self, X, centroids, std_list):
        RBF_list = []
        for x in X:
            RBF_list.append([self.get_rbf(x, c, s) for (c, s) in zip(centroids, std_list)])
        return np.array(RBF_list)


    def fit(self):

        self.centroids, self.std_list = kmeans(self.X, self.k, 1000)

        if not self.std_from_clusters:
            dMax = np.max([get_distance(c1, c2) for c1 in self.centroids for c2 in self.centroids])
            self.std_list = np.repeat(dMax / np.sqrt(2 * self.k), self.k)

        RBF_X = self.get_rbf_as_list(self.X, self.centroids, self.std_list)

        self.w = np.linalg.pinv(RBF_X.T @ RBF_X) @ RBF_X.T @ self.convert_to_one_hot(self.y, self.number_of_classes)

        RBF_list_tst = self.get_rbf_as_list(self.tX, self.centroids, self.std_list)

        self.pred_ty = RBF_list_tst @ self.w

        self.pred_ty = np.array([np.argmax(x) for x in self.pred_ty])

        diff = self.pred_ty - self.ty

        print('Accuracy: ', len(np.where(diff == 0)[0]) / len(diff))

**Βήμα 3ο**

Τρέχουμε τη συνάρτηση όπου το RBFNN γίνεται fit. Αρχικά τρέχει ο αλγόριθμος k-means για να βρούμε τα κέντρα και την τυπική απόκλιση των clusters. Στη συνέχεια, μπορούμε να χρησιμοποιήσουμε το ίδιο beta για όλα τα clusters


---
***Ερώτηση 3***

Με ποιούς τρόπους μπορούμε να υπολογίσουμε τα βάρη στα hidden layers; Ποιες διαφορές υπάρχουν στους τρόπους αυτούς; Ποιος παρέχει τη καλύτερη δυνατή λύση;

**Aπάντηση 3**

Μπορούμε είτε με Least Squares είτε με Gradient Desent (Batch,Stockastic κτλπ) με backpropagation. H διαφορά τους είναι ότι για την Least Squares δεν απαιτούνται πολλαπλές ενημερώσεις ούτε εποχές σε αντίθεση με τον Gradient Desent, παρόλα αυτά η μέθοδος Least Squares μπορεί να είναι πολύ απαιτητική σε μνήμη και υπολογιστική ισχύ όταν τα δεδομένα μας είναι πάρα πολλά και έχουμε και αντίστοιχα πολλά βάρη να καθορίσουμε ενώ σε μειωμένες διαστάσεις είναι αρκετά γρήγορή και οικονομική. Από τις 2 μεθόδους η Least Squares εγγυάται την βέλτιστη λύση.


**Βήμα 4ο**

Σπάμε το dataset του MNIST σε training και testing και αφήνουμε το RBFNN να βγάλει το τελικό αποτέλεσμα.

---

***Ερώτηση 4***

Επιχειρήστε να αλλάξετε την τιμή του k σε 500 και στη συνέχεια σε 1000. Τι παρατηρείτε ως προς την εκτέλεση;

**Απάντηση 4**

Αυξάνει πάρα πολύ ο χρόνος καθώς με k=500 προσεγγίζει την 1h30min με ενώ για k=1000 προσεγγίζει την 2h30min σε  αντίθεση με 10min οταν θέτουμε k=10, έχουμε όμως και μεγάλη αύξηση του accuracy αφού από 66% που φτάνουμε με k=10 , φτάνουμε το 95% στις δυο προηγούμενες περιπτώσεις.

In [22]:
##################################

data = np.load('/content/drive/MyDrive/Files/mnist_data.npy').astype(float)

train_y = data[0:2500,0]
train_x = data[0:2500,1:]

test_y = data[2500:2800,0]
test_x = data[2500:2800,1:]

RBF_CLASSIFIER = RBF(train_x, train_y, test_x, test_y, num_of_classes=10,
                     k=10, std_from_clusters=False)

RBF_CLASSIFIER.fit()

K-MEANS:  29688
K-MEANS:  10256
K-MEANS:  8099
K-MEANS:  4374
K-MEANS:  2685
K-MEANS:  2096
K-MEANS:  1118
K-MEANS:  1268
K-MEANS:  1090
K-MEANS:  777
K-MEANS:  280
K-MEANS:  603
K-MEANS:  339
K-MEANS:  363
K-MEANS:  521
K-MEANS:  218
K-MEANS:  56
K-MEANS:  331
K-MEANS:  7
K-MEANS:  102
K-MEANS:  31
K-MEANS:  159
K-MEANS:  108
K-MEANS:  323
K-MEANS:  59
K-MEANS:  31
K-MEANS:  138
K-MEANS:  21
K-MEANS:  2
K-MEANS:  25
K-MEANS:  6
K-MEANS:  56
K-MEANS:  56
K-MEANS:  3
K-MEANS:  32
K-MEANS:  40
K-MEANS:  47
K-MEANS:  7
K-MEANS:  44
K-MEANS:  9
K-MEANS:  0
Accuracy:  0.6666666666666666


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
